# Get invoice **status**
This endpoint is used to retrieve invoice integration or notification status in archive.

### Endpoint steps
1. Get a token from your credentials by calling the endpoint **_Account/getToken_**;
2. Get an invoice status in archive calling the endpoint **_OutboundFinancialDocument/{documentId}_**;

#### Response structure from server
When a request is well formed and the authentication data is correct the system responds with a message envelope as follows: 

```Javascript
{
	"CorrelationId": "<GUID>", /* for correlation purposes */
	"IsValid": true,           /* false in case of erros */
	"Errors": [],              /* if empty is a good signal */
	"Data": "<Service Response Data>"   /* the data retuned ex: token, invoice status, dependent on the endpoint called */
}
```

## 1. Get a token (Account/getToken)
Get a token from your account credentials using endpoint api/Account/getToken  
Check [here](./get-token) how to obtain an authentication token. 

## 2. Get a Document status by DocumentId (OutboundFinancialDocument/{documentId})

### 2.1 Request structure
In the endpoint url you need to supply the outboundfinancialdocumentId received

#### OutboundFinancialDocumentId
- **Description**: You need to supply the id of the invoice.
- **Required**: Yes
- **Type**: string
- **Example**: "3fa85f64-5717-4562-b3fc-2c963f66afa6"

### 2.2 Request example

In [18]:
# SIN endpoint url for retrieving inforfation on invoice previously sent
outboundFinancialDocumentId = "fc5e547d-8537-4e05-97d5-1159c62efd6f"
service_url = "https://" + server_base_adress + "/api/OutboundFinancialDocument/" + outboundFinancialDocumentId
print (service_url)

https://dcn-solution.saphety.com/Dcn.Sandbox.WebApi/api/OutboundFinancialDocument/fc5e547d-8537-4e05-97d5-1159c62efd6f


### 2.3 Call the endpoint to get the document
**Get** the invoice status by the **Document id** (“OutboundFinancialDocumentId“) using endpoint _**OutboundFinancialDocument/:id**_

In [13]:
# build the request
headers = {
    'Authorization': 'bearer ' + token
    }
# POST request to send the invoice
response = requests.request("GET", service_url, headers=headers)

# formating the response to json for visualization purposes only
json_response = json.loads(response.text)
print(json.dumps(json_response["Data"], indent=4))

{
    "Id": "fc5e547d-8537-4e05-97d5-1159c62efd6f",
    "VirtualOperatorCode": null,
    "CompanyIntlVatCode": "PT507641230",
    "DocumentType": "INVOICE",
    "DocumentDate": "2020-12-31 00:00:00",
    "DocumentNumber": "FTSCN 2020/00000346",
    "ReceiverIntlVatCode": "PT507957547",
    "ReceiverName": "SAPHETY LEVEL - TRUSTED SERVICES, S.A",
    "DocumentStatus": "Final",
    "DocumentSource": "Integration",
    "DocumentLink": "",
    "DocumentTotal": 11.07,
    "CurrencyCode": "EUR",
    "NotificationStatus": "Delivered",
    "IntegrationStatus": "Received",
    "IntegrationDate": "2021-01-03 00:00:00",
    "LastUpdateDate": "2021-01-03 00:00:00",
    "AuthorId": "442ad0fe-74e8-41c7-9d3f-fd009534b26c",
    "Errors": null,
    "CreationDate": "2021-01-01 00:00:00"
}


There are 2 types of documents in the archive:

* Electronic invoice
* PDF invoice

To query the status of a document in archive, it is necessary to consider 2 concepts of status:

* **Integration status** - indicates the current integration status that a document has in archive
* **Notification status** - indicates the current email notification status that a document has in archive

The ***integration status*** of a document in archive is returned at ***"IntegrationStatus"*** parameter, see the possible values returned:

* **NotIntegrated** - document not sent to receiver entity for integration
* **Not_Sent** - document not sent for receiver entity
* **Error** - document with errors, not sent to receiver entity for integration
* **Sent** - document sent to receiver entity for integration
* **Received** - document sent to receiver entity for integration, receiver entity has confirmed its reception
* **Rejected** - document sent to receiver entity for integration, receiver entity is rejecting it
* **Paid** - document sent to receiver entity for integration, receiver entity is confirming it as paid

The ***notification status*** of a document in archive is returned at ***"NotificationStatus"*** parameter, see the possible values returned:

* **NoEmails** - document without any email notification
* **Sent** - document with at least one email notification sent
* **Delivered** - document with at least one email notification delivered
* **Read** - document with at least one email notification read
* **Error** - document with at all email notifications in error

**[Important note_1]** If an electronic invoice is sent to an entity outside Saphety network, it will not be sent to receiver entity for integration, so the **integration status** will always be ***"NotIntegrated"***.

In this scenario the electronic invoice may have also a **notification status** *(Sent, Delivered, Read or Error)* if the CIUS-PT XML has the tag BT-49 filled in with the receiver entity's email address (ex: <cbc:EndpointID schemeID="EM">eee@hh.pt</cbc:EndpointID>), otherwise the document will have **integration status** as ***"NotIntegrated"*** and **notification status** as ***"NoEmails"***.

**[Important note_2]** If an electronic invoice is sent to a receiver entity of a network with interoperability, but the issuer does not have an approved connection to this network, it will not be sent to receiver entity for integration, so the **integration status** will always be ***"NotIntegrated"***.

In this scenario the electronic invoice may have also a **notification status** *(Sent, Delivered, Read or Error)* if the CIUS-PT XML has the tag BT-49 filled in with the receiver entity's email address (ex: <cbc:EndpointID schemeID="EM">eee@hh.pt</cbc:EndpointID>), otherwise the document will have **integration status** as ***"NotIntegrated"*** and **notification status** as ***"NoEmails"***.

**[Important note_3]** A PDF invoice has always the **integration status** as ***"NotIntegrated"***.

If the PDF invoice was signed, stored and sent by email to the receiver entity, the **notification status** will return the status "***Sent***", "***Delivered***", "***Read***" or "***Error***".

If the PDF invoice was signed, stored and not sent by email to the receiver entity, the **notification status** will return the status ***"NoEmails"***.